# Pulling IPL 2017 Season Tweets

## Abstract

Now, we have converted our SQL database that we created previously to NoSQL using the ProjectB database. We have combined the ‘Player’ and ‘Team’ tables into a single ‘Player_Team’ collection. Also, we have combined the ‘Venue’, ‘Match’ and ‘Result; tables into a single ‘Match_Result’ collection. The NoSQL database uses MongoDB. Then we have extracted tweets from twitter which contain relevant hash tags for the IPL 2017 season. Then, there is an explanation of the trending and popular tweets and users and how it provides an insight on the season. 

### Installing the Tweepy library for accessing the Twitter API.

In [60]:
pip install tweepy

Note: you may need to restart the kernel to use updated packages.


### Importing Libraries 

In [61]:
import os
import pandas as pd
import tweepy as tw

### Reading the Excel file 'playerdata' 

In [62]:
playerdata=pd.read_excel('C:/Users/Abhishek/Desktop/playerdata.xlsx')

### Printing the Excel file 'playerdata'  

In [63]:
playerdata

,Player_ID,Player_Name,Player_Handle,Date_of_Birth,Team_ID,Country,Batting_Style,Bowling_Style,Role,Player_Captain_ID
0,32,Shane Watson,ShaneRWatson33,1981-06-17,2,Australia,Right-hand bat,Right-arm fast-medium,Captain,8
1,162,Chris Gayle,henrygayle,1979-09-21,2,West Indies,Left-hand bat,Right-arm offbreak,Player,8
2,382,YS Chahal,yuzi_chahal,1990-07-23,2,India,Right-hand bat,Legbreak googly,Player,8
3,31,Yusuf Pathan,iamyusufpathan,1982-11-17,1,India,Right-hand bat,Right-arm offbreak,Player,40
4,40,Gautam Gambhir,GautamGambhir,1981-10-14,1,India,Left-hand bat,Legbreak,Captain,40
5,46,Robin Aiyuda Uthappa,robbieuthappa,1985-11-11,1,India,Right-hand bat,Right-arm medium,Keeper,40
6,96,MK Pandey,im_manishpandey,1989-09-10,1,India,Right-hand bat,Right-arm medium,Player,40
7,479,CR Woakes,chriswoakes,1989-03-02,1,England,ÿRight-hand bat,Right-arm fast-medium,Player,40
8,305,GJ Maxwell,Gmaxi_32,1988-10-14,3,Australia,Right-hand bat,Right-arm offbreak,Captain,305
9,320,DA Miller,DavidMillerSA12,1989-06-10,3,South Africa,Left-hand bat,Right-arm offbreak,Player,305


### Defining the API keys and Access tokens

In [64]:
api_key= 'oXwTHAuHz9efkni6qnw5ukOd4'
api_secret_key= 'tKT5kyAvBHmuX2JydmwkgJH1lvMYYxXrmpgnyNfcCzGWBq1GAW'
access_token= '4443869112-xESKHkPgBmwWrLqAAerchhmdb5HOjnHdYDHjZRd'
access_token_secret= 'Zk9MtZN09VbyDx5cYReiuMZFdPjBoVToMqGNMrkTLArAQ'

In [65]:
auth = tw.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

### Getting the Player Twitter Users

In [69]:
playtwit=playerdata['Player_Handle'].values.tolist()[:10]

### Printing the Player Twitter Handles from our Database

In [71]:
playtwit

['ShaneRWatson33',
 'henrygayle',
 'yuzi_chahal',
 'iamyusufpathan',
 'GautamGambhir',
 'robbieuthappa',
 'im_manishpandey',
 'chriswoakes',
 'Gmaxi_32',
 'DavidMillerSA12']

In [78]:
playertwitterusers=[]
for i in range(0,len(playtwit)):
    try:
        user = api.get_user(playtwit[i])
        print(user)
        playertwitterusers.append(playtwit[i])
    except:
        pass
    
for user in playertwitterusers:
    for tweet in api.user_timeline(id=user, result_type="recent", count="number"):
        print(tweet.created_at, tweet.text) 

User(_api=<tweepy.api.API object at 0x00000233F9514888>, _json={'id': 2220638376, 'id_str': '2220638376', 'name': 'Shane Watson', 'screen_name': 'ShaneRWatson33', 'location': 'Australia', 'profile_location': {'id': '3f14ce28dc7c4566', 'url': 'https://api.twitter.com/1.1/geo/id/3f14ce28dc7c4566.json', 'place_type': 'unknown', 'name': 'Australia', 'full_name': 'Australia', 'country_code': '', 'country': '', 'contained_within': [], 'bounding_box': None, 'attributes': {}}, 'description': 'Family is everything!!! T20 Nomad. Living life to the fullest 😊😊', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 1043121, 'friends_count': 202, 'listed_count': 644, 'created_at': 'Fri Nov 29 01:57:05 +0000 2013', 'favourites_count': 159, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': True, 'statuses_count': 520, 'lang': None, 'status': {'created_at': 'Tue Mar 31 07:53:36 +0000 2020', 'id': 1244895605865906176, 'id_str': '1244895605

In [90]:
users = ["ShaneRWatson33", "henrygayle", "yuzi_chahal",
 "iamyusufpathan",
 "GautamGambhir",
 "robbieuthappa",
 "im_manishpandey",
 "chriswoakes",
 "Gmaxi_32",
 "DavidMillerSA12"]

In [91]:
compdict = {}
def compSearch(query):
    tweets = []
    count = 500
    
    searchresult = api.search(q=query,count=count,since="2020-02-20", until="2020-04-09", lang="en")
    for tweet in searchresult:
        tags = []
        hashtags = (tweet._json["entities"]["hashtags"])
        username = tweet._json["user"]['user_name']
        for tag in hashtags:
            tags.append(tag["text"])
        if len(tags) == 0:
            tags = ""
        tweetobj ={
                "player_name":query,
                "created_at":tweet.created_at,
                "username":username,
                "id":tweet.id,
                "text":tweet.text,
                "Retweetcount":tweet.retweet_count,
                "FavoriteCount":tweet.favorite_count,
                "tags":tags
        }
        tweets.append(tweetobj)
    compdict[query] = tweets

In [92]:
for startup in users:
    compSearch(startup)

compdict

{'ShaneRWatson33': [{'company_name': 'ShaneRWatson33',
   'created_at': datetime.datetime(2020, 4, 8, 20, 28, 48),
   'screenname': 'Rohiitrock',
   'id': 1247984764591562752,
   'text': '@rajasthanroyals @ajinkyarahane88 @josbuttler @IamSanjuSamson @benstokes38 @ShaneRWatson33 @stevesmith49 @JUnadkat… https://t.co/YBjR79cfAh',
   'retweet': 0,
   'favCount': 0,
   'tags': ''},
  {'company_name': 'ShaneRWatson33',
   'created_at': datetime.datetime(2020, 4, 8, 19, 36, 44),
   'screenname': 'lokesh0119',
   'id': 1247971661321613312,
   'text': '@rajasthanroyals @ajinkyarahane88 @josbuttler @IamSanjuSamson @benstokes38 @ShaneRWatson33 @stevesmith49 @JUnadkat… https://t.co/uc3eoVompw',
   'retweet': 0,
   'favCount': 0,
   'tags': ''},
  {'company_name': 'ShaneRWatson33',
   'created_at': datetime.datetime(2020, 4, 8, 19, 35, 47),
   'screenname': 'AdityaTanjore',
   'id': 1247971419192823808,
   'text': '@IshantBishnoi1 @rajasthanroyals @ajinkyarahane88 @josbuttler @IamSanjuSamson @bens

### Questions Answered 

What are tags are associated with a person, place or thing?

There are tags such as SRH, CSK, IPL2017, WhistlePodu, CSKisthebest, CSKvsMI, etc. which are associated with players from those respective teams.



What social media users are like other social media users in your domain?

Players like Robin Uthappa and Gautam Gambhir have similar tags, posts, and hashtags as they are from the same team.



 What people, places or things are popular in your domain?
 
 Teams like mumbai Indians are popular as they are the winners of the IPL 2017 season. So it makes sense that they are popular.

 What people, places or things are trending in your domain? (A trend is popularity
over time.)

The team names and slogans such as 'Whistle Podu' for the Chennai Super Kings are trending.

## Citations 

•	https://stackoverflow.com/questions/35813854/how-to-join-multiple-collections-with-lookup-in-mongodb
•	https://www.pythoncentral.io/introduction-to-tweepy-twitter-for-python/
•	https://www.tweepy.org/
•	https://www.twitter.com/
•	https://developer.twitter.com/en


## Conclusion

We have successfully pulled player tweets and drawn conclusions by looking at trends and figures of the tweets.